### Agents
Agents combine language models with tools to create systems that can reason about tasks, decide which tools to use, and iteratively work towards solutions.

In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

In [ ]:
from langchain_core.tools import tool


@tool
def fake_weather_api(city: str) -> str:
    """
    Check the weather in a specified city.

    Args:
        city (str): The name of the city where you want to check the weather.

    Returns:
        str: A description of the current weather in the specified city.
    """
    return "Sunny, 22°C"

tools = [fake_weather_api]

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    "openai:gpt-4o-mini",
    tools=tools,
    system_prompt="You are a helpful assistant"
)

In [ ]:
agent.invoke({"messages": [{"role": "user", "content": "How are you?"}]})

In [ ]:
agent.invoke({"messages": [{"role": "user", "content": "How is the weather in Berlin?"}]})

### Agents with Memory

In [ ]:
from langgraph.checkpoint.memory import InMemorySaver  

agent = create_agent(
    "openai:gpt-4o-mini",
    tools=tools,
    checkpointer=InMemorySaver(),  
)

In [ ]:
agent.invoke({"messages": [{"role": "user", "content": "Hi my name is Bob?"}]}, {"configurable": {"thread_id": "1"}})

In [ ]:
agent.invoke({"messages": [{"role": "user", "content": "Do you remember my name?"}]}, {"configurable": {"thread_id": "1"}},)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain_core.messages import AIMessage
from langgraph.checkpoint.memory import InMemorySaver

basic_model = ChatOpenAI(model="gpt-4o-mini")

@wrap_model_call
def cap_after_three_messages(request: ModelRequest, handler) -> ModelResponse | AIMessage:
    message_count = len(request.state["messages"])
    if message_count >= 3:
        return AIMessage(content="Iam tired")
    return handler(request)

agent = create_agent(
    model=basic_model,
    tools=tools,
    middleware=[cap_after_three_messages],
    checkpointer=InMemorySaver(),
)

In [ ]:
agent.invoke({"messages": [{"role": "user", "content": "Hi my name is Bob?"}]},
             {"configurable": {"thread_id": "1"}})
agent.invoke({"messages": [{"role": "user", "content": "Tell me a joke"}]},
             {"configurable": {"thread_id": "1"}})
agent.invoke({"messages": [{"role": "user", "content": "Another one"}]},
             {"configurable": {"thread_id": "1"}})